# 6.0 LangChain 1.0 Middleware & Modern Agent Patterns

This notebook showcases the **brand new features in LangChain 1.0**, including:

- **`create_agent`**: The new unified API for building agents
- **Middleware System**: Hook into the agent loop for human-in-the-loop, summarization, and custom logic
- **Structured Outputs with ToolStrategy**: Constrain agent responses to specific schemas
- **Model String Format**: Simplified model initialization

These features represent a major evolution in how we build LLM applications!

## Setup

In [ ]:
%pip install -qU langchain>=1.0.0 langgraph>=1.0.0
%pip install -qU langchain-openai langchain-anthropic
%pip install -qU langchain-tavily

In [ ]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("TAVILY_API_KEY")
_set_env("LANGCHAIN_API_KEY")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langchain-1.0-middleware-demo"

## Part 1: The New `create_agent` API

LangChain 1.0 introduces `create_agent` - a unified way to create agents that replaces:
- `AgentExecutor`
- `create_react_agent`
- `create_tool_calling_agent`
- `langgraph.prebuilt.create_react_agent`

### Key Benefits:
1. **Simpler API**: Single function for all agent types
2. **Model Strings**: Use `"openai:gpt-4o-mini"` instead of instantiating classes
3. **Built-in Middleware**: Human-in-the-loop, summarization, etc.
4. **Returns a Graph**: Full LangGraph `CompiledStateGraph` under the hood

In [ ]:
from langchain.agents import create_agent
from langchain_core.tools import tool


@tool
def get_weather(location: str) -> str:
    """Get the current weather for a location."""
    # Simulated weather data
    weather_data = {
        "san francisco": "62°F, foggy",
        "new york": "75°F, sunny",
        "london": "55°F, rainy",
        "tokyo": "82°F, humid",
    }
    return weather_data.get(location.lower(), f"Weather data unavailable for {location}")


@tool
def calculate(expression: str) -> str:
    """Evaluate a mathematical expression."""
    try:
        # Safe evaluation of math expressions
        result = eval(expression, {"__builtins__": {}}, {})
        return str(result)
    except Exception as e:
        return f"Error: {e}"


# Create an agent with the new API - note the model string format!
agent = create_agent(
    model="openai:gpt-4o-mini",  # Provider:model format
    tools=[get_weather, calculate],
    system_prompt="You are a helpful assistant. Be concise in your responses."
)

print(f"Agent type: {type(agent)}")

In [ ]:
# Invoke the agent - uses messages format
result = agent.invoke({
    "messages": [{"role": "user", "content": "What's the weather in Tokyo?"}]
})

print(result["messages"][-1].content)

In [ ]:
# Multi-turn conversation
result = agent.invoke({
    "messages": [
        {"role": "user", "content": "What's the weather in San Francisco?"},
        {"role": "assistant", "content": "The weather in San Francisco is 62°F and foggy."},
        {"role": "user", "content": "What about New York? And how much warmer is it there?"}
    ]
})

print(result["messages"][-1].content)

In [ ]:
# Stream the response
print("Streaming response:")
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "Calculate (25 * 4) + 17"}]},
    stream_mode="values"
):
    chunk["messages"][-1].pretty_print()
    print("-" * 50)

## Part 2: Web Search Agent with TavilySearch

Let's create a research agent that can search the web using the new `langchain-tavily` package.

In [ ]:
from langchain_tavily import TavilySearch

# Create the Tavily search tool with configuration
tavily_search = TavilySearch(
    max_results=5,
    topic="general",
    include_answer=True,  # Get a direct answer when possible
)

# Create a research agent
research_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[tavily_search],
    system_prompt="""You are a research assistant. 
    When answering questions:
    1. Use the search tool to find current information
    2. Cite your sources with URLs
    3. Be concise but comprehensive"""
)

In [ ]:
# Research a current topic
result = research_agent.invoke({
    "messages": [{"role": "user", "content": "What are the key features of LangChain 1.0?"}]
})

print(result["messages"][-1].content)

## Part 3: Middleware System

LangChain 1.0's middleware system lets you hook into the agent loop at three points:

| Hook | Timing | Use Cases |
|------|--------|----------|
| `before_model` | Before each model call | Logging, rate limiting, context injection |
| `after_model` | After each model call | Response filtering, metrics collection |
| `modify_model_request` | Before model call | Modify tools, prompts, or model settings |

### Built-in Middleware:
- **SummarizationMiddleware**: Compress long conversations
- **HumanInTheLoopMiddleware**: Require approval before tool execution

### 3.1 Summarization Middleware

Automatically compresses conversation history when it gets too long.

In [ ]:
from langchain.agents.middleware import SummarizationMiddleware

# Create agent with summarization middleware
summarizing_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_weather, calculate],
    system_prompt="You are a helpful assistant.",
    middleware=[
        SummarizationMiddleware(
            model="openai:gpt-4o-mini",  # Model for summarization
            trigger=("tokens", 4000),     # Trigger when > 4000 tokens
            keep=("messages", 20),        # Keep last 20 messages
        ),
    ],
)

print("Agent with summarization created!")

In [ ]:
# Have a conversation (summarization triggers automatically)
result = summarizing_agent.invoke({
    "messages": [
        {"role": "user", "content": "What's the weather in all the cities you know about?"}
    ]
})

print(result["messages"][-1].content)

### 3.2 Custom Middleware

Let's create a custom middleware that logs all tool calls.

In [ ]:
from langchain.agents.middleware import AgentMiddleware
from datetime import datetime


class LoggingMiddleware(AgentMiddleware):
    """Middleware that logs all model calls and tool invocations."""
    
    def __init__(self):
        self.call_count = 0
        self.tool_calls = []
    
    def before_model(self, state, config):
        """Called before each model invocation."""
        self.call_count += 1
        print(f"\n[{datetime.now().strftime('%H:%M:%S')}] Model call #{self.call_count}")
        print(f"  Messages: {len(state.get('messages', []))}")
        return None  # Return None to continue normally
    
    def after_model(self, state, config):
        """Called after each model invocation."""
        last_message = state.get('messages', [])[-1] if state.get('messages') else None
        if last_message and hasattr(last_message, 'tool_calls') and last_message.tool_calls:
            for tc in last_message.tool_calls:
                self.tool_calls.append(tc['name'])
                print(f"  Tool called: {tc['name']}")
        return None
    
    def get_stats(self):
        return {
            "total_calls": self.call_count,
            "tool_calls": self.tool_calls
        }

In [ ]:
# Create logging middleware instance
logging_middleware = LoggingMiddleware()

# Create agent with custom middleware
logged_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_weather, calculate],
    system_prompt="You are a helpful assistant.",
    middleware=[logging_middleware],
)

# Run some queries
result = logged_agent.invoke({
    "messages": [{"role": "user", "content": "What's the weather in Tokyo and what's 100 * 15?"}]
})

print(f"\nFinal answer: {result['messages'][-1].content}")
print(f"\nStats: {logging_middleware.get_stats()}")

## Part 4: Structured Outputs with ToolStrategy

LangChain 1.0 integrates structured outputs directly into the agent loop using `response_format`.

In [ ]:
from pydantic import BaseModel, Field
from langchain.agents.structured_output import ToolStrategy


class WeatherReport(BaseModel):
    """Structured weather report."""
    location: str = Field(description="The location queried")
    temperature: str = Field(description="Current temperature")
    conditions: str = Field(description="Weather conditions")
    recommendation: str = Field(description="What to wear or bring")


# Create agent with structured output
structured_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_weather],
    system_prompt="You are a weather assistant. Always provide practical recommendations.",
    response_format=ToolStrategy(WeatherReport),  # Constrain output!
)

In [ ]:
# Get structured weather report
result = structured_agent.invoke({
    "messages": [{"role": "user", "content": "What's the weather in San Francisco?"}]
})

# The response is now structured!
response = result["response"]
print(f"Location: {response.location}")
print(f"Temperature: {response.temperature}")
print(f"Conditions: {response.conditions}")
print(f"Recommendation: {response.recommendation}")

## Part 5: Persistence and Memory

Add persistence to maintain conversation state across invocations.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# Create a checkpointer for memory
memory = MemorySaver()

# Create agent with memory
memory_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_weather, calculate],
    system_prompt="You are a helpful assistant. Remember details from our conversation.",
    checkpointer=memory,  # Enable persistence!
)

In [ ]:
# Configuration with thread_id for session management
config = {"configurable": {"thread_id": "user-123"}}

# First message
result = memory_agent.invoke(
    {"messages": [{"role": "user", "content": "My name is Alice and I live in Tokyo."}]},
    config=config
)
print(f"Response 1: {result['messages'][-1].content}")

In [ ]:
# Follow-up - the agent remembers!
result = memory_agent.invoke(
    {"messages": [{"role": "user", "content": "What's the weather where I live?"}]},
    config=config
)
print(f"Response 2: {result['messages'][-1].content}")

In [ ]:
# Yet another follow-up
result = memory_agent.invoke(
    {"messages": [{"role": "user", "content": "What's my name again?"}]},
    config=config
)
print(f"Response 3: {result['messages'][-1].content}")

## Part 6: Multi-Provider Support

The model string format makes it easy to switch providers!

In [ ]:
# Create agents with different providers using the same API

# OpenAI agent
openai_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_weather],
    system_prompt="You are a weather assistant."
)

# You can also use Anthropic (if you have ANTHROPIC_API_KEY set)
# anthropic_agent = create_agent(
#     model="anthropic:claude-sonnet-4-5-20250929",
#     tools=[get_weather],
#     system_prompt="You are a weather assistant."
# )

print("Agents created successfully!")

In [ ]:
# Test OpenAI agent
result = openai_agent.invoke({
    "messages": [{"role": "user", "content": "Weather in London please!"}]
})
print(f"OpenAI: {result['messages'][-1].content}")

## Part 7: Complete Example - Research Assistant

Let's put it all together with a research assistant that:
- Searches the web
- Has conversation memory
- Logs all actions
- Returns structured responses

In [ ]:
from pydantic import BaseModel, Field
from typing import List


class ResearchSummary(BaseModel):
    """Structured research summary."""
    topic: str = Field(description="The research topic")
    key_findings: List[str] = Field(description="Main findings (3-5 bullet points)")
    sources: List[str] = Field(description="URLs of sources used")
    confidence: str = Field(description="Confidence level: high/medium/low")


# Create comprehensive research agent
research_assistant = create_agent(
    model="openai:gpt-4o-mini",
    tools=[tavily_search],
    system_prompt="""You are a research assistant. When researching:
    1. Search for multiple perspectives
    2. Verify information from multiple sources
    3. Note your confidence level based on source quality
    4. Always cite your sources""",
    checkpointer=MemorySaver(),
    middleware=[LoggingMiddleware()],
)

In [ ]:
# Research a topic
config = {"configurable": {"thread_id": "research-session-1"}}

result = research_assistant.invoke(
    {"messages": [{"role": "user", "content": "What are the main differences between LangChain and LangGraph?"}]},
    config=config
)

print(result["messages"][-1].content)

In [ ]:
# Follow-up question
result = research_assistant.invoke(
    {"messages": [{"role": "user", "content": "Based on what you found, which should I use for a simple chatbot?"}]},
    config=config
)

print(result["messages"][-1].content)

## Summary: LangChain 1.0 Key Changes

### What's New

| Feature | Pre-v1 | LangChain 1.0 |
|---------|--------|---------------|
| Agent Creation | `AgentExecutor`, `create_react_agent` | `create_agent()` |
| Model Init | `ChatOpenAI(model="gpt-4o-mini")` | `"openai:gpt-4o-mini"` |
| Middleware | Not available | `SummarizationMiddleware`, custom middleware |
| Structured Output | Separate chains | `response_format=ToolStrategy(Schema)` |
| Tavily Search | `TavilySearchResults` (community) | `TavilySearch` (langchain-tavily) |
| State Schema | Pydantic, dataclass | TypedDict only |

### Migration Checklist

1. ✅ Replace `AgentExecutor` with `create_agent()`
2. ✅ Use model string format: `"openai:gpt-4o-mini"`
3. ✅ Install `langchain-tavily` for search tools
4. ✅ Use `TypedDict` for state schemas
5. ✅ Add middleware for cross-cutting concerns
6. ✅ Use `response_format` for structured outputs

### Resources

- [LangChain 1.0 Migration Guide](https://docs.langchain.com/oss/python/migrate/langchain-v1)
- [LangChain 1.0 Blog Post](https://blog.langchain.com/langchain-langgraph-1dot0/)
- [Built-in Middleware Docs](https://docs.langchain.com/oss/python/langchain/middleware/built-in)